Review Analysis of Amazon Dataset

In [1]:
!pip install nltk

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, ENGLISH_STOP_WORDS

[nltk_data] Downloading package punkt to /Users/nisha/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
#Creating Stop Words
stop_words_list = list(ENGLISH_STOP_WORDS)+["Review","Reviews", "Amazon", "amazon","reviews","review"]

In [4]:
df = pd.read_csv('amazon_review_full_csv/train.csv', delimiter=',', header=None)

In [5]:
df.head()

,0,1,2
0,3,more like funchuck,Gave this to my dad for a gag gift after direc...
1,5,Inspiring,I hope a lot of people hear this cd. We need m...
2,5,The best soundtrack ever to anything.,I'm reading a lot of reviews saying that this ...
3,4,Chrono Cross OST,The music of Yasunori Misuda is without questi...
4,5,Too good to be true,Probably the greatest soundtrack in history! U...


In [6]:
df.columns = ['Rating',"Review_Title", "Review"]

In [7]:
df.shape

(3000000, 3)

In [8]:
df.head()

,Rating,Review_Title,Review
0,3,more like funchuck,Gave this to my dad for a gag gift after direc...
1,5,Inspiring,I hope a lot of people hear this cd. We need m...
2,5,The best soundtrack ever to anything.,I'm reading a lot of reviews saying that this ...
3,4,Chrono Cross OST,The music of Yasunori Misuda is without questi...
4,5,Too good to be true,Probably the greatest soundtrack in history! U...


<h1>DATA WRANGLING</h1>

# 1. Check for missing values

In [9]:
df.isnull().mean()*100

Rating          0.000000
Review_Title    0.006267
Review          0.000000
dtype: float64

In [10]:
df.isnull().sum()

Rating            0
Review_Title    188
Review            0
dtype: int64

# Out of 3000000, there are 188 missing values , Since the missing values are in title, it may not make difference in finding the analysis, so this can be ignored.

# 2. Checking DataTyes are correct.

In [11]:
data_types = df.dtypes
print(data_types)

Rating           int64
Review_Title    object
Review          object
dtype: object


In [12]:
# converting the Columns to appropriate dataType for Review_Title and Review
df["Review_Title"] = df["Review_Title"].astype(str)
df["Review"] = df["Review"].astype(str)

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000000 entries, 0 to 2999999
Data columns (total 3 columns):
 #   Column        Dtype 
---  ------        ----- 
 0   Rating        int64 
 1   Review_Title  object
 2   Review        object
dtypes: int64(1), object(2)
memory usage: 68.7+ MB


In [14]:
df['Review_Title'] = df['Review_Title'].astype(str)
df["Review"] = df["Review"].astype(str)

In [15]:
df.dtypes

Rating           int64
Review_Title    object
Review          object
dtype: object

In [16]:
print(type(df['Review_Title'][0]))  # Should print <class 'str'>
print(type(df['Review'][0]))        # Should print <class 'str'>

<class 'str'>
<class 'str'>


In [17]:
# checking Duplicate values

duplicated = df[df.duplicated()]
print(duplicated)

Empty DataFrame
Columns: [Rating, Review_Title, Review]
Index: []


In [18]:
# There are no Duplicate Reviews

In [19]:
# Checking Duplicates based on specific column
dup1 = df[df.duplicated(subset=['Rating','Review_Title'])]
dup2 = df[df.duplicated(subset=['Rating','Review'])]

In [20]:
dup1.sort_values(by='Review_Title')

,Rating,Review_Title,Review
858263,3,!,Our Lady Of Weight Loss has Miraculous and Mot...
1250365,5,!,"This is an incredibly well-researched, well-wr..."
1126169,5,!,This product is awesome! I have friends who ha...
1818343,4,!,The ZZ Top cover is worth the entrance fee alo...
2060684,4,!,Book arrived in stated condition and prior to ...
...,...,...,...
1301070,2,zzzzzzzzzzzzzzzzzzzzzz,I bought this book as a text required for a cl...
2280312,4,~,Good product. Pads that came with it are crap ...
2817352,5,~,"I'm not an avid anime fan, but my friend is. S..."
2456111,1,¡Decepcionante!,Aunque el desempeño de la orquesta es excelent...


In [21]:
df.groupby(df.columns.tolist()).size().reset_index(name='Duplicate_Count').sort_values(by='Duplicate_Count', ascending=False)

,Rating,Review_Title,Review,Duplicate_Count
0,1,!,This book is completely out of date. The probl...,1
1999993,4,Good read on fiction not not-fiction accounts,"If this were a real story, WOW, but alas it is...",1
1999995,4,Good read on the strength of teamwork and working,together toward a common goal. Nice reference ...,1
1999996,4,Good read overall,Overall this was a good book. It was a very ea...,1
1999997,4,Good read that encourages the reader to think,Generally well written story - with the except...,1
...,...,...,...,...
1000001,2,Shotgun Wedding,Problem here is that it isn't Barber's book. C...,1
1000002,2,Shotgun stock kit,It was very difficult to install. It was not l...,1
1000003,2,Shotgun writing style didn't work for me,"If I had to sum this book up in one word, it w...",1
1000004,2,"Shots are too distant, but okay for watching w...",If you are looking for an exercise video that ...,1


In [22]:
df[df['Review_Title']=='3 books on learning frontpage 2003']

,Rating,Review_Title,Review


In [23]:
print(dup1)

         Rating     Review_Title  \
95            2     Not the best   
103           1  I got fooled...   
202           2    Disappointing   
297           3   Mixed feelings   
555           2     disappointed   
...         ...              ...   
2999986       3       Incomplete   
2999987       2  Not the Best...   
2999992       2   Too simplistic   
2999995       1    Don't do it!!   
2999998       3   Hard to clean!   

                                                    Review  
95       I agree, the CDs are a much needed help, since...  
103      I did not know this was the audio cd of the ga...  
202      I bought this book because I am a nonogram (al...  
297      Some beautiful scenery early on and nice roman...  
555      I think this is a case of my expectations bein...  
...                                                    ...  
2999986  The three films in this set are excellent, abs...  
2999987  I bought this CD expecting it to be the same t...  
2999992  While Mr.

In [24]:
dup2.head()

,Rating,Review_Title,Review
1072,1,Wouldn't stay up,The band was far too thin and would roll down ...
7002,2,lesson in the depravity of man,"As I opened this book, I thought to myself ""Wo..."
12273,3,Shows Its Age,"Unfortunately, ""20,000 Leagues Under the Sea"" ..."
15487,1,Dumb adaptation that does not try to be even r...,Lackluster adaptation of the vertigo comic boo...
15531,1,What a letdown.,"Shameful attempt at a hybrid of horror, action..."


In [25]:
# doubt - are these duplicates ?

# DATA WRANGLING  - NLP

In [26]:
df.Rating.value_counts()

Rating
3    600000
5    600000
4    600000
1    600000
2    600000
Name: count, dtype: int64

In [27]:
df.head()

,Rating,Review_Title,Review
0,3,more like funchuck,Gave this to my dad for a gag gift after direc...
1,5,Inspiring,I hope a lot of people hear this cd. We need m...
2,5,The best soundtrack ever to anything.,I'm reading a lot of reviews saying that this ...
3,4,Chrono Cross OST,The music of Yasunori Misuda is without questi...
4,5,Too good to be true,Probably the greatest soundtrack in history! U...


In [28]:
#1. Rule Based Labelling
def rating_sentiment(rating):
    if rating ==3:
        return "Neutral"  
    elif rating > 3:
        return "Positive" 
    else:
        return "Negative"
    
df["Rating_Sentiment"] = df["Rating"].apply(rating_sentiment)

In [29]:
df.head()

,Rating,Review_Title,Review,Rating_Sentiment
0,3,more like funchuck,Gave this to my dad for a gag gift after direc...,Neutral
1,5,Inspiring,I hope a lot of people hear this cd. We need m...,Positive
2,5,The best soundtrack ever to anything.,I'm reading a lot of reviews saying that this ...,Positive
3,4,Chrono Cross OST,The music of Yasunori Misuda is without questi...,Positive
4,5,Too good to be true,Probably the greatest soundtrack in history! U...,Positive


In [30]:
df['Rating_Sentiment'].value_counts()

Rating_Sentiment
Positive    1200000
Negative    1200000
Neutral      600000
Name: count, dtype: int64

In [31]:
# Processing all the strings to lower case
df['Review_Title'] = df['Review_Title'].str.lower()
df['Review'] = df['Review'].str.lower()

In [32]:
df.head()

,Rating,Review_Title,Review,Rating_Sentiment
0,3,more like funchuck,gave this to my dad for a gag gift after direc...,Neutral
1,5,inspiring,i hope a lot of people hear this cd. we need m...,Positive
2,5,the best soundtrack ever to anything.,i'm reading a lot of reviews saying that this ...,Positive
3,4,chrono cross ost,the music of yasunori misuda is without questi...,Positive
4,5,too good to be true,probably the greatest soundtrack in history! u...,Positive


In [33]:
# Rearranging the column
column_to_move = df.pop('Rating_Sentiment')
df.insert(1,'Rating_Sentiment',column_to_move)

In [34]:
df.head()

,Rating,Rating_Sentiment,Review_Title,Review
0,3,Neutral,more like funchuck,gave this to my dad for a gag gift after direc...
1,5,Positive,inspiring,i hope a lot of people hear this cd. we need m...
2,5,Positive,the best soundtrack ever to anything.,i'm reading a lot of reviews saying that this ...
3,4,Positive,chrono cross ost,the music of yasunori misuda is without questi...
4,5,Positive,too good to be true,probably the greatest soundtrack in history! u...


In [35]:
df['Review_Count'] = df['Review'].str.len()
df['Review_Title_count'] = df['Review_Title'].str.len()

In [36]:
df.head()

,Rating,Rating_Sentiment,Review_Title,Review,Review_Count,Review_Title_count
0,3,Neutral,more like funchuck,gave this to my dad for a gag gift after direc...,93,18
1,5,Positive,inspiring,i hope a lot of people hear this cd. we need m...,204,9
2,5,Positive,the best soundtrack ever to anything.,i'm reading a lot of reviews saying that this ...,470,37
3,4,Positive,chrono cross ost,the music of yasunori misuda is without questi...,393,16
4,5,Positive,too good to be true,probably the greatest soundtrack in history! u...,377,19


In [37]:
!pip install textblob

In [38]:
df.head()

,Rating,Rating_Sentiment,Review_Title,Review,Review_Count,Review_Title_count
0,3,Neutral,more like funchuck,gave this to my dad for a gag gift after direc...,93,18
1,5,Positive,inspiring,i hope a lot of people hear this cd. we need m...,204,9
2,5,Positive,the best soundtrack ever to anything.,i'm reading a lot of reviews saying that this ...,470,37
3,4,Positive,chrono cross ost,the music of yasunori misuda is without questi...,393,16
4,5,Positive,too good to be true,probably the greatest soundtrack in history! u...,377,19


In [39]:
from nltk import word_tokenize

In [40]:
df['title_word_tokenize_count'] = df['Review_Title'].apply(word_tokenize)
df['Review_word_tokenize_count'] = df['Review'].apply(word_tokenize)

In [41]:
df.head()

,Rating,Rating_Sentiment,Review_Title,Review,Review_Count,Review_Title_count,title_word_tokenize_count,Review_word_tokenize_count
0,3,Neutral,more like funchuck,gave this to my dad for a gag gift after direc...,93,18,"[more, like, funchuck]","[gave, this, to, my, dad, for, a, gag, gift, a..."
1,5,Positive,inspiring,i hope a lot of people hear this cd. we need m...,204,9,[inspiring],"[i, hope, a, lot, of, people, hear, this, cd, ..."
2,5,Positive,the best soundtrack ever to anything.,i'm reading a lot of reviews saying that this ...,470,37,"[the, best, soundtrack, ever, to, anything, .]","[i, 'm, reading, a, lot, of, reviews, saying, ..."
3,4,Positive,chrono cross ost,the music of yasunori misuda is without questi...,393,16,"[chrono, cross, ost]","[the, music, of, yasunori, misuda, is, without..."
4,5,Positive,too good to be true,probably the greatest soundtrack in history! u...,377,19,"[too, good, to, be, true]","[probably, the, greatest, soundtrack, in, hist..."


In [42]:
df = df.rename(columns={'Review_Count':'Review_str_len', 'Review_Title_count':'Title_str_len', 'title_word_tokenize_count':'title_word_tokenize', 'Review_word_tokenize_count':'Review_word_tokenize'})

In [43]:
df.head()

,Rating,Rating_Sentiment,Review_Title,Review,Review_str_len,Title_str_len,title_word_tokenize,Review_word_tokenize
0,3,Neutral,more like funchuck,gave this to my dad for a gag gift after direc...,93,18,"[more, like, funchuck]","[gave, this, to, my, dad, for, a, gag, gift, a..."
1,5,Positive,inspiring,i hope a lot of people hear this cd. we need m...,204,9,[inspiring],"[i, hope, a, lot, of, people, hear, this, cd, ..."
2,5,Positive,the best soundtrack ever to anything.,i'm reading a lot of reviews saying that this ...,470,37,"[the, best, soundtrack, ever, to, anything, .]","[i, 'm, reading, a, lot, of, reviews, saying, ..."
3,4,Positive,chrono cross ost,the music of yasunori misuda is without questi...,393,16,"[chrono, cross, ost]","[the, music, of, yasunori, misuda, is, without..."
4,5,Positive,too good to be true,probably the greatest soundtrack in history! u...,377,19,"[too, good, to, be, true]","[probably, the, greatest, soundtrack, in, hist..."


In [44]:
df['title_wtoken_cnt'] = df['title_word_tokenize'].apply(len)
df['Review_wtoken_cnt'] = df['Review_word_tokenize'].apply(len)

In [45]:
df.head()

,Rating,Rating_Sentiment,Review_Title,Review,Review_str_len,Title_str_len,title_word_tokenize,Review_word_tokenize,title_wtoken_cnt,Review_wtoken_cnt
0,3,Neutral,more like funchuck,gave this to my dad for a gag gift after direc...,93,18,"[more, like, funchuck]","[gave, this, to, my, dad, for, a, gag, gift, a...",3,24
1,5,Positive,inspiring,i hope a lot of people hear this cd. we need m...,204,9,[inspiring],"[i, hope, a, lot, of, people, hear, this, cd, ...",1,44
2,5,Positive,the best soundtrack ever to anything.,i'm reading a lot of reviews saying that this ...,470,37,"[the, best, soundtrack, ever, to, anything, .]","[i, 'm, reading, a, lot, of, reviews, saying, ...",7,101
3,4,Positive,chrono cross ost,the music of yasunori misuda is without questi...,393,16,"[chrono, cross, ost]","[the, music, of, yasunori, misuda, is, without...",3,76
4,5,Positive,too good to be true,probably the greatest soundtrack in history! u...,377,19,"[too, good, to, be, true]","[probably, the, greatest, soundtrack, in, hist...",5,81


In [46]:
import string
punctuation = set(string.punctuation)

def remove_punctuation_stop_words(reviews):
    punctuation_removed = [wordlist for wordlist in reviews if wordlist not in punctuation]
    filtered_tokens = [word for word in punctuation_removed if word not in stop_words_list]
    return " ".join(filtered_tokens)

In [47]:
df["review_punc_stop_words_removed"] = df["Review_word_tokenize"].apply(remove_punctuation_stop_words)

In [48]:
df.head()

,Rating,Rating_Sentiment,Review_Title,Review,Review_str_len,Title_str_len,title_word_tokenize,Review_word_tokenize,title_wtoken_cnt,Review_wtoken_cnt,review_punc_stop_words_removed
0,3,Neutral,more like funchuck,gave this to my dad for a gag gift after direc...,93,18,"[more, like, funchuck]","[gave, this, to, my, dad, for, a, gag, gift, a...",3,24,gave dad gag gift directing `` nunsense '' got...
1,5,Positive,inspiring,i hope a lot of people hear this cd. we need m...,204,9,[inspiring],"[i, hope, a, lot, of, people, hear, this, cd, ...",1,44,hope lot people hear cd need strong positive v...
2,5,Positive,the best soundtrack ever to anything.,i'm reading a lot of reviews saying that this ...,470,37,"[the, best, soundtrack, ever, to, anything, .]","[i, 'm, reading, a, lot, of, reviews, saying, ...",7,101,'m reading lot saying best 'game soundtrack fi...
3,4,Positive,chrono cross ost,the music of yasunori misuda is without questi...,393,16,"[chrono, cross, ost]","[the, music, of, yasunori, misuda, is, without...",3,76,music yasunori misuda question close second gr...
4,5,Positive,too good to be true,probably the greatest soundtrack in history! u...,377,19,"[too, good, to, be, true]","[probably, the, greatest, soundtrack, in, hist...",5,81,probably greatest soundtrack history usually '...


In [49]:
df["review_removed_cnt"] = df["review_punc_stop_words_removed"].apply(len)

In [50]:
df.head()

,Rating,Rating_Sentiment,Review_Title,Review,Review_str_len,Title_str_len,title_word_tokenize,Review_word_tokenize,title_wtoken_cnt,Review_wtoken_cnt,review_punc_stop_words_removed,review_removed_cnt
0,3,Neutral,more like funchuck,gave this to my dad for a gag gift after direc...,93,18,"[more, like, funchuck]","[gave, this, to, my, dad, for, a, gag, gift, a...",3,24,gave dad gag gift directing `` nunsense '' got...,57
1,5,Positive,inspiring,i hope a lot of people hear this cd. we need m...,204,9,[inspiring],"[i, hope, a, lot, of, people, hear, this, cd, ...",1,44,hope lot people hear cd need strong positive v...,140
2,5,Positive,the best soundtrack ever to anything.,i'm reading a lot of reviews saying that this ...,470,37,"[the, best, soundtrack, ever, to, anything, .]","[i, 'm, reading, a, lot, of, reviews, saying, ...",7,101,'m reading lot saying best 'game soundtrack fi...,255
3,4,Positive,chrono cross ost,the music of yasunori misuda is without questi...,393,16,"[chrono, cross, ost]","[the, music, of, yasunori, misuda, is, without...",3,76,music yasunori misuda question close second gr...,270
4,5,Positive,too good to be true,probably the greatest soundtrack in history! u...,377,19,"[too, good, to, be, true]","[probably, the, greatest, soundtrack, in, hist...",5,81,probably greatest soundtrack history usually '...,210


In [51]:
!pip install threadpoolctl

In [52]:
# Using Stemmer and WordLemmetizer
from nltk.stem import PorterStemmer, WordNetLemmatizer

porterStemmer = PorterStemmer()
wordNetLemmetizer = WordNetLemmatizer()

In [53]:
def porter_stemmer_tokenizer(word_tokenized):
    return [porterStemmer.stem(tokens) for tokens in word_tokenized]

def wnet_lemmatizer_tokenizer(word_tokenized):
    return [wordNetLemmetizer.lemmatize(tokens) for tokens in word_tokenized]

In [62]:
# Porter Stemming    
df["porterStemmer"] = porter_stemmer_tokenizer(df["Review_word_tokenize"].astype(str))

In [63]:
df.head()

,Rating,Rating_Sentiment,Review_Title,Review,Review_str_len,Title_str_len,title_word_tokenize,Review_word_tokenize,title_wtoken_cnt,Review_wtoken_cnt,review_punc_stop_words_removed,review_removed_cnt,porterStemmer,WordNet_Lemmatizer
0,3,Neutral,more like funchuck,gave this to my dad for a gag gift after direc...,93,18,"[more, like, funchuck]","[gave, this, to, my, dad, for, a, gag, gift, a...",3,24,gave dad gag gift directing `` nunsense '' got...,57,"['gave', 'this', 'to', 'my', 'dad', 'for', 'a'...","[gave, this, to, my, dad, for, a, gag, gift, a..."
1,5,Positive,inspiring,i hope a lot of people hear this cd. we need m...,204,9,[inspiring],"[i, hope, a, lot, of, people, hear, this, cd, ...",1,44,hope lot people hear cd need strong positive v...,140,"['i', 'hope', 'a', 'lot', 'of', 'people', 'hea...","[i, hope, a, lot, of, people, hear, this, cd, ..."
2,5,Positive,the best soundtrack ever to anything.,i'm reading a lot of reviews saying that this ...,470,37,"[the, best, soundtrack, ever, to, anything, .]","[i, 'm, reading, a, lot, of, reviews, saying, ...",7,101,'m reading lot saying best 'game soundtrack fi...,255,"['i', ""'m"", 'reading', 'a', 'lot', 'of', 'revi...","[i, 'm, reading, a, lot, of, review, saying, t..."
3,4,Positive,chrono cross ost,the music of yasunori misuda is without questi...,393,16,"[chrono, cross, ost]","[the, music, of, yasunori, misuda, is, without...",3,76,music yasunori misuda question close second gr...,270,"['the', 'music', 'of', 'yasunori', 'misuda', '...","[the, music, of, yasunori, misuda, is, without..."
4,5,Positive,too good to be true,probably the greatest soundtrack in history! u...,377,19,"[too, good, to, be, true]","[probably, the, greatest, soundtrack, in, hist...",5,81,probably greatest soundtrack history usually '...,210,"['probably', 'the', 'greatest', 'soundtrack', ...","[probably, the, greatest, soundtrack, in, hist..."


In [64]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /Users/nisha/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [57]:
# WordNet Lemmatizer
from nltk.stem import WordNetLemmatizer
df["WordNet_Lemmatizer"] = df["Review_word_tokenize"].apply(wnet_lemmatizer_tokenizer)

In [58]:
df.head()

,Rating,Rating_Sentiment,Review_Title,Review,Review_str_len,Title_str_len,title_word_tokenize,Review_word_tokenize,title_wtoken_cnt,Review_wtoken_cnt,review_punc_stop_words_removed,review_removed_cnt,porterStemmer,WordNet_Lemmatizer
0,3,Neutral,more like funchuck,gave this to my dad for a gag gift after direc...,93,18,"[more, like, funchuck]","[gave, this, to, my, dad, for, a, gag, gift, a...",3,24,gave dad gag gift directing `` nunsense '' got...,57,"[gave, thi, to, my, dad, for, a, gag, gift, af...","[gave, this, to, my, dad, for, a, gag, gift, a..."
1,5,Positive,inspiring,i hope a lot of people hear this cd. we need m...,204,9,[inspiring],"[i, hope, a, lot, of, people, hear, this, cd, ...",1,44,hope lot people hear cd need strong positive v...,140,"[i, hope, a, lot, of, peopl, hear, thi, cd, .,...","[i, hope, a, lot, of, people, hear, this, cd, ..."
2,5,Positive,the best soundtrack ever to anything.,i'm reading a lot of reviews saying that this ...,470,37,"[the, best, soundtrack, ever, to, anything, .]","[i, 'm, reading, a, lot, of, reviews, saying, ...",7,101,'m reading lot saying best 'game soundtrack fi...,255,"[i, 'm, read, a, lot, of, review, say, that, t...","[i, 'm, reading, a, lot, of, review, saying, t..."
3,4,Positive,chrono cross ost,the music of yasunori misuda is without questi...,393,16,"[chrono, cross, ost]","[the, music, of, yasunori, misuda, is, without...",3,76,music yasunori misuda question close second gr...,270,"[the, music, of, yasunori, misuda, is, without...","[the, music, of, yasunori, misuda, is, without..."
4,5,Positive,too good to be true,probably the greatest soundtrack in history! u...,377,19,"[too, good, to, be, true]","[probably, the, greatest, soundtrack, in, hist...",5,81,probably greatest soundtrack history usually '...,210,"[probabl, the, greatest, soundtrack, in, histo...","[probably, the, greatest, soundtrack, in, hist..."


In [59]:
#Using tfidf - vectorizer
tfidf_vect = TfidfVectorizer(max_features=100, stop_words=stop_words_list, ngram_range=(1,2), min_df=30)
tfidf_vect.fit(df.review_punc_stop_words_removed)
tfidf_X = tfidf_vect.transform(df.review_punc_stop_words_removed)

tfidf_df = pd.DataFrame(tfidf_X.toarray(), columns= tfidf_vect.get_feature_names_out())
print(tfidf_df.head())


   actually  album  author  bad      best    better  big      bit  book  \
0       0.0    0.0     0.0  0.0  0.000000  0.000000  0.0  0.00000   0.0   
1       0.0    0.0     0.0  0.0  0.000000  0.000000  0.0  0.00000   0.0   
2       0.0    0.0     0.0  0.0  0.225909  0.000000  0.0  0.26126   0.0   
3       0.0    0.0     0.0  0.0  0.000000  0.000000  0.0  0.00000   0.0   
4       0.0    0.0     0.0  0.0  0.000000  0.316646  0.0  0.00000   0.0   

   books  ...   ve  want  watch  way  work  works     worth  written  year  \
0    0.0  ...  0.0   0.0    0.0  0.0   0.0    0.0  0.000000      0.0   0.0   
1    0.0  ...  0.0   0.0    0.0  0.0   0.0    0.0  0.000000      0.0   0.0   
2    0.0  ...  0.0   0.0    0.0  0.0   0.0    0.0  0.251830      0.0   0.0   
3    0.0  ...  0.0   0.0    0.0  0.0   0.0    0.0  0.000000      0.0   0.0   
4    0.0  ...  0.0   0.0    0.0  0.0   0.0    0.0  0.383421      0.0   0.0   

      years  
0  0.000000  
1  0.000000  
2  0.240946  
3  0.000000  
4  0.00000

Concatenating Dataframe with Tfidf dataframe

In [60]:
df_tf = pd.concat([df.reset_index(drop=True),tfidf_df.reset_index(drop=True)],axis=1)
print(df_tf.head())

   Rating Rating_Sentiment                           Review_Title  \
0       3          Neutral                     more like funchuck   
1       5         Positive                              inspiring   
2       5         Positive  the best soundtrack ever to anything.   
3       4         Positive                       chrono cross ost   
4       5         Positive                    too good to be true   

                                              Review  Review_str_len  \
0  gave this to my dad for a gag gift after direc...              93   
1  i hope a lot of people hear this cd. we need m...             204   
2  i'm reading a lot of reviews saying that this ...             470   
3  the music of yasunori misuda is without questi...             393   
4  probably the greatest soundtrack in history! u...             377   

   Title_str_len                             title_word_tokenize  \
0             18                          [more, like, funchuck]   
1              9

In [69]:
ratings_map = {
    1: "Bad",
    2: "Below Average",
    3: "Average",
    4: "Above average",
    5: "Good"
}


In [70]:
df_tf["likert_scale"] = df_tf["Rating"].apply(lambda rate: ratings_map.get(rate, None))  

In [71]:
df_tf.head()

,Rating,Rating_Sentiment,Review_Title,Review,Review_str_len,Title_str_len,title_word_tokenize,Review_word_tokenize,title_wtoken_cnt,Review_wtoken_cnt,...,want,watch,way,work,works,worth,written,year,years,likert_scale
0,3,Neutral,more like funchuck,gave this to my dad for a gag gift after direc...,93,18,"[more, like, funchuck]","[gave, this, to, my, dad, for, a, gag, gift, a...",3,24,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,Average
1,5,Positive,inspiring,i hope a lot of people hear this cd. we need m...,204,9,[inspiring],"[i, hope, a, lot, of, people, hear, this, cd, ...",1,44,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,Good
2,5,Positive,the best soundtrack ever to anything.,i'm reading a lot of reviews saying that this ...,470,37,"[the, best, soundtrack, ever, to, anything, .]","[i, 'm, reading, a, lot, of, reviews, saying, ...",7,101,...,0.0,0.0,0.0,0.0,0.0,0.251830,0.0,0.0,0.240946,Good
3,4,Positive,chrono cross ost,the music of yasunori misuda is without questi...,393,16,"[chrono, cross, ost]","[the, music, of, yasunori, misuda, is, without...",3,76,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,Above average
4,5,Positive,too good to be true,probably the greatest soundtrack in history! u...,377,19,"[too, good, to, be, true]","[probably, the, greatest, soundtrack, in, hist...",5,81,...,0.0,0.0,0.0,0.0,0.0,0.383421,0.0,0.0,0.000000,Good


#Saving the Dataframe as CSV

In [72]:
df_tf.to_csv('data_wrangled_reviews_amazon.csv', index=False)